Cho đến giờ, chúng ta đã thảo luận về cách xử lý dữ liệu cũng như cách xây dựng, đào tạo và thử nghiệm các mô hình học sâu. Tuy nhiên, tại một thời điểm nào đó, chúng tôi hy vọng sẽ đủ hài lòng với các mô hình đã học mà chúng tôi sẽ muốn lưu kết quả để sử dụng sau này trong các ngữ cảnh khác nhau (thậm chí có thể đưa ra dự đoán khi triển khai). Ngoài ra, khi chạy một quy trình đào tạo dài, cách tốt nhất là lưu định kỳ các kết quả trung gian (điểm kiểm tra) để đảm bảo rằng chúng tôi không mất giá trị tính toán trong vài ngày nếu chúng tôi vấp phải dây nguồn của máy chủ. Vì vậy, đã đến lúc tìm hiểu cách tải và lưu trữ cả vectơ trọng số riêng lẻ và toàn bộ mô hình. Phần này giải quyết cả hai vấn đề.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

# 6.6.1. Loading và Saving Tensor

Đối với các tenxơ riêng lẻ, chúng ta có thể gọi trực tiếp các hàm `load` và `save` để đọc và ghi chúng tương ứng. Cả hai chức năng đều yêu cầu chúng tôi cung cấp tên và `save` yêu cầu đầu vào là biến được lưu.

In [2]:
x = torch.arange(4)
torch.save(x, 'x-file')

Bây giờ chúng ta có thể đọc dữ liệu từ tệp được lưu trữ trở lại bộ nhớ.

In [3]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

Chúng ta có thể lưu trữ một danh sách các tenxơ và đọc lại chúng vào bộ nhớ.

In [4]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

Thậm chí chúng ta có thể viết và đọc một từ điển ánh xạ từ các chuỗi đến các tenxơ. Điều này thuận tiện khi chúng ta muốn đọc hoặc ghi tất cả các trọng số trong một mô hình.

In [5]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

# 6.6.2. Loading and Saving thông số mô hình

Việc lưu các vectơ trọng số riêng lẻ (hoặc các tenxơ khác) là hữu ích, nhưng sẽ rất tẻ nhạt nếu chúng ta muốn lưu (và sau đó tải) toàn bộ mô hình. Xét cho cùng, chúng ta có thể có hàng trăm nhóm thông số rải rác khắp nơi. Vì lý do này, khung học sâu cung cấp các chức năng tích hợp để tải và lưu toàn bộ mạng. Một chi tiết quan trọng cần lưu ý là điều này lưu các tham số mô hình chứ không phải toàn bộ mô hình. Ví dụ: nếu chúng tôi có MLP 3 lớp, chúng tôi cần chỉ định kiến ​​trúc riêng. Lý do cho điều này là bản thân các mô hình có thể chứa mã tùy ý, do đó chúng không thể được đánh số tự nhiên. Vì vậy, để khôi phục một mô hình, chúng ta cần tạo kiến ​​trúc bằng mã và sau đó tải các tham số từ đĩa. Hãy bắt đầu với MLP quen thuộc của chúng ta.

In [30]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.LazyLinear(256)
    self.output = nn.LazyLinear(10)

  def forward(self, x):
    return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.0399, -0.2167, -0.1513,  ...,  0.2095, -0.0620,  0.1744],
                      [ 0.1475,  0.0790, -0.2067,  ..., -0.2166,  0.0103, -0.1407],
                      [ 0.0548,  0.0513,  0.1175,  ..., -0.1218,  0.1777, -0.0839],
                      ...,
                      [ 0.1123, -0.1863,  0.1180,  ..., -0.2068, -0.0705, -0.1864],
                      [ 0.2023,  0.0134, -0.1631,  ...,  0.1258,  0.0777, -0.2234],
                      [-0.1739,  0.1357,  0.1811,  ..., -0.1831, -0.0864,  0.1021]])),
             ('hidden.bias',
              tensor([ 0.1606,  0.0514,  0.1713,  0.0453, -0.1621,  0.0519, -0.1876, -0.1140,
                       0.0497,  0.0616, -0.0619,  0.2085,  0.0622,  0.0805,  0.0650,  0.1001,
                      -0.1270, -0.0401, -0.0405, -0.1237,  0.0212, -0.0677,  0.0022, -0.1901,
                      -0.1643, -0.0652, -0.1665,  0.1109,  0.1602, -0.2090, -0.0933,  0.0628,
                       0.1633,

Tiếp theo, chúng tôi lưu trữ các tham số của mô hình dưới dạng tệp có tên “mlp.params”.

In [31]:
torch.save(net.state_dict(), 'mlp.params')

Để khôi phục mô hình, chúng tôi khởi tạo một bản sao của mô hình MLP ban đầu. Thay vì khởi tạo ngẫu nhiên các tham số mô hình, chúng tôi đọc trực tiếp các tham số được lưu trữ trong tệp.

In [32]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

Vì cả hai trường hợp đều có các tham số mô hình giống nhau nên kết quả tính toán của cùng một đầu vào X sẽ giống nhau. Hãy kiểm chứng điều này.

In [33]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])